In [ ]:
# | default_exp _docusaurus_helper

# Docusaurus Helper

In [ ]:
# | export

import itertools
import re
import types
from inspect import Signature, getmembers, isclass, isfunction, signature
from pathlib import Path
from typing import *
from urllib.parse import urljoin

import yaml
from docstring_parser import parse
from docstring_parser.common import DocstringParam, DocstringRaises, DocstringReturns
from nbdev.config import get_config
from nbdev_mkdocs.mkdocs import (
    _add_all_submodules,
    _get_api_summary,
    _import_all_members,
    _import_functions_and_classes,
    _import_submodules,
)

/Users/harishm/.pyenv/versions/3.10.4/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
from tempfile import TemporaryDirectory

import pytest
from pydantic import BaseModel

In [ ]:
# | export


def _format_docstring_sections(
    items: Union[List[DocstringParam], List[DocstringReturns], List[DocstringRaises]],
    keyword: str,
) -> str:
    """Format a list of docstring sections

    Args:
        items: A list of DocstringParam objects
        keyword: The type of section to format (e.g. 'Parameters', 'Returns', 'Exceptions')

    Returns:
        The formatted docstring.
    """
    formatted_docstring = ""
    if len(items) > 0:
        formatted_docstring += f"**{keyword}**:\n"
        for item in items:
            if keyword == "Parameters":
                formatted_docstring += f"- `{item.arg_name}`: {item.description}\n"  # type: ignore
            elif keyword == "Exceptions":
                formatted_docstring += f"- `{item.type_name}`: {item.description}\n"
            else:
                formatted_docstring += f"- {item.description}\n"
        formatted_docstring = f"{formatted_docstring}\n"
    return formatted_docstring

In [ ]:
fixture = parse(
    """
This is a docstring for a sample function.

It can contain multiple lines and can include *markdown* syntax.

Args:
    name: name of the person
    age: age of the person
    
Returns:
    A formatted string

Raises:
    ValueError: If name is not a string
    TypeError: If name is not a string
"""
)

actual = _format_docstring_sections(fixture.params, "Parameters")
expected = """**Parameters**:
- `name`: name of the person
- `age`: age of the person

"""

print(actual)
assert actual == expected

**Parameters**:
- `name`: name of the person
- `age`: age of the person




In [ ]:
actual = _format_docstring_sections(fixture.many_returns, "Returns")
expected = """**Returns**:
- A formatted string

"""

print(actual)
assert actual == expected

**Returns**:
- A formatted string




In [ ]:
actual = _format_docstring_sections(fixture.raises, "Exceptions")
expected = """**Exceptions**:
- `ValueError`: If name is not a string
- `TypeError`: If name is not a string

"""

print(actual)
assert actual == expected

**Exceptions**:
- `ValueError`: If name is not a string
- `TypeError`: If name is not a string




In [ ]:
# | export


def _docstring_to_markdown(docstring: str) -> str:
    """Converts a docstring to a markdown-formatted string.

    Args:
        docstring: The docstring to convert.

    Returns:
        The markdown-formatted docstring.
    """
    parsed_docstring = parse(docstring)
    formatted_docstring = f"{parsed_docstring.short_description}\n\n"
    formatted_docstring += (
        f"{parsed_docstring.long_description}\n\n"
        if parsed_docstring.long_description
        else ""
    )
    formatted_docstring += _format_docstring_sections(
        parsed_docstring.params, "Parameters"
    )
    formatted_docstring += _format_docstring_sections(
        parsed_docstring.many_returns, "Returns"
    )
    formatted_docstring += _format_docstring_sections(
        parsed_docstring.raises, "Exceptions"
    )

    return formatted_docstring

In [ ]:
fixture = """This is a docstring for a sample function."""

expected = """This is a docstring for a sample function.

"""

actual = _docstring_to_markdown(fixture)
print(actual)

assert actual == expected

This is a docstring for a sample function.




In [ ]:
fixture = """This is a docstring for a sample function.

Args:
    name: name of the person
    age: age of the person
"""

expected = """This is a docstring for a sample function.

**Parameters**:
- `name`: name of the person
- `age`: age of the person

"""

actual = _docstring_to_markdown(fixture)
print(actual)

assert actual == expected

This is a docstring for a sample function.

**Parameters**:
- `name`: name of the person
- `age`: age of the person




In [ ]:
fixture = """
This is a docstring for a sample function.

It can contain multiple lines and can include *markdown* syntax.

Args:
    name: name of the person
    age: age of the person
    
Returns:
    A formatted string

Raises:
    ValueError: If name is not a string
    TypeError: If name is not a string
"""

expected = """This is a docstring for a sample function.

It can contain multiple lines and can include *markdown* syntax.

**Parameters**:
- `name`: name of the person
- `age`: age of the person

**Returns**:
- A formatted string

**Exceptions**:
- `ValueError`: If name is not a string
- `TypeError`: If name is not a string

"""

actual = _docstring_to_markdown(fixture)
print(actual)

assert actual == expected

This is a docstring for a sample function.

It can contain multiple lines and can include *markdown* syntax.

**Parameters**:
- `name`: name of the person
- `age`: age of the person

**Returns**:
- A formatted string

**Exceptions**:
- `ValueError`: If name is not a string
- `TypeError`: If name is not a string




In [ ]:
# | export


def _get_submodules(module_name: str) -> List[str]:
    """Get a list of all submodules contained within the module.

    Args:
        module_name: The name of the module to retrieve submodules from

    Returns:
        A list of submodule names within the module
    """
    members = _import_all_members(module_name)
    members_with_submodules = _add_all_submodules(members)
    members_with_submodules_str: List[str] = [
        x[:-1] if x.endswith(".") else x for x in members_with_submodules
    ]
    return members_with_submodules_str

In [ ]:
module_name = "fastkafka"
members_with_submodules = _get_submodules(module_name)
members_with_submodules

['fastkafka',
 'fastkafka.EventMetadata',
 'fastkafka.FastKafka',
 'fastkafka.KafkaEvent',
 'fastkafka.testing',
 'fastkafka.testing.ApacheKafkaBroker',
 'fastkafka.testing.LocalRedpandaBroker',
 'fastkafka.testing.Tester']

In [ ]:
# | export


def _load_submodules(
    module_name: str, members_with_submodules: List[str]
) -> List[Union[types.FunctionType, Type[Any]]]:
    """Load the given submodules from the module.

    Args:
        module_name: The name of the module whose submodules to load
        members_with_submodules: A list of submodule names to load

    Returns:
        A list of imported submodule objects.
    """
    submodules = _import_submodules(module_name)
    members: List[Tuple[str, Union[types.FunctionType, Type[Any]]]] = list(
        itertools.chain(*[_import_functions_and_classes(m) for m in submodules])
    )
    names = [
        y
        for x, y in members
        if f"{y.__module__}.{y.__name__}" in members_with_submodules
    ]
    return names

In [ ]:
module_name = "fastkafka"
members_with_submodules = _get_submodules(module_name)
symbols = _load_submodules(module_name, members_with_submodules)
symbols

[fastkafka.EventMetadata,
 fastkafka.FastKafka,
 fastkafka.KafkaEvent,
 fastkafka.testing.ApacheKafkaBroker,
 fastkafka.testing.LocalRedpandaBroker,
 fastkafka.testing.Tester]

In [ ]:
# | export


def _convert_union_to_optional(annotation_str: str) -> str:
    """Convert the 'Union[Type1, Type2, ..., NoneType]' to 'Optional[Type1, Type2, ...]' in the given annotation string

    Args:
        annotation_str: The type annotation string to convert.

    Returns:
        The converted type annotation string.
    """
    pattern = r"Union\[(.*)?,\s*NoneType\s*\]"
    match = re.search(pattern, annotation_str)
    if match:
        union_type = match.group(1)
        optional_type = f"Optional[{union_type}]"
        return re.sub(pattern, optional_type, annotation_str)
    else:
        return annotation_str

In [ ]:
fixtures = [
    {
        "input": "arg_1: Union[int, NoneType] = 80",
        "expected": "arg_1: Optional[int] = 80",
    },
    {
        "input": "arg_1: Union[Dict[str, str], NoneType]",
        "expected": "arg_1: Optional[Dict[str, str]]",
    },
    {
        "input": "arg_1: Union[Dict[str, str], str]",
        "expected": "arg_1: Union[Dict[str, str], str]",
    },
    {
        "input": "arg_1: str",
        "expected": "arg_1: str",
    },
    {
        "input": "arg_1: bool = False",
        "expected": "arg_1: bool = False",
    },
    {
        "input": "prefix: str = 'to_'",
        "expected": "prefix: str = 'to_'",
    },
]

for fixture in fixtures:
    actual = _convert_union_to_optional(fixture["input"])
    print(actual)
    assert actual == fixture["expected"]

arg_1: Optional[int] = 80
arg_1: Optional[Dict[str, str]]
arg_1: Union[Dict[str, str], str]
arg_1: str
arg_1: bool = False
prefix: str = 'to_'


In [ ]:
# | export


def _get_arg_list_with_signature(_signature: Signature) -> str:
    """Converts a function's signature into a string representation of its argument list.

    Args:
        _signature (signature): The signature object for the function to convert.

    Returns:
        str: A string representation of the function's argument list.
    """
    arg_list = []
    for param in _signature.parameters.values():
        arg_list.append(_convert_union_to_optional(str(param)))

    return ", ".join(arg_list)

In [ ]:
def fixture_function(
    arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80
) -> str:
    pass


_signature = signature(fixture_function)

expected = (
    "arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80"
)
actual = _get_arg_list_with_signature(_signature)

print(actual)
assert actual == expected

arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80


In [ ]:
def fixture_function(arg_1: str, arg_2) -> None:
    pass


_signature = signature(fixture_function)

expected = "arg_1: str, arg_2"
actual = _get_arg_list_with_signature(_signature)

print(actual)
assert actual == expected

arg_1: str, arg_2


In [ ]:
# | export


def _get_symbol_definition(symbol: Union[types.FunctionType, Type[Any]]) -> str:
    """Return the definition of a given symbol.

    Args:
        symbol: A function or method object to get the definition for.

    Returns:
        A string representing the function definition
    """
    _signature = signature(symbol)
    arg_list = _get_arg_list_with_signature(_signature)
    ret_val = ""

    if isfunction(symbol):
        ret_val = f"### `{symbol.__name__}`" + f" {{#{symbol.__name__.strip('_')}}}\n\n"
        ret_val = ret_val + f"`def {symbol.__name__}({arg_list})"
        if _signature.return_annotation and "inspect._empty" not in str(
            _signature.return_annotation
        ):
            if isinstance(_signature.return_annotation, type):
                ret_val = ret_val + f" -> {_signature.return_annotation.__name__}`\n"
            else:
                ret_val = ret_val + f" -> {_signature.return_annotation}`\n"

        else:
            ret_val = ret_val + " -> None`\n"

    return ret_val

In [ ]:
TestCallable = Callable[[BaseModel], Union[Awaitable[None], None]]


def fixture_function(arg_1: str) -> TestCallable:
    pass


actual = _get_symbol_definition(fixture_function)
print(actual)
assert (
    "`def fixture_function(arg_1: str) -> typing.Callable[[pydantic.main.BaseModel]"
    in actual
)

### `fixture_function` {#fixture_function}

`def fixture_function(arg_1: str) -> typing.Callable[[pydantic.main.BaseModel], typing.Optional[typing.Awaitable[NoneType]]]`



In [ ]:
def fixture_function(arg_1: str, arg_2) -> None:
    pass


actual = _get_symbol_definition(fixture_function)
expected = "### `fixture_function` {#fixture_function}\n\n`def fixture_function(arg_1: str, arg_2) -> None`\n"

print(actual)
assert actual == expected

### `fixture_function` {#fixture_function}

`def fixture_function(arg_1: str, arg_2) -> None`



In [ ]:
def fixture_function(arg_1: str, arg_2) -> int:
    pass


actual = _get_symbol_definition(fixture_function)
expected = "### `fixture_function` {#fixture_function}\n\n`def fixture_function(arg_1: str, arg_2) -> int`\n"

print(actual)
assert actual == expected

### `fixture_function` {#fixture_function}

`def fixture_function(arg_1: str, arg_2) -> int`



In [ ]:
def fixture_function(arg_1: str, arg_2) -> "Tester":
    pass


actual = _get_symbol_definition(fixture_function)
expected = "### `fixture_function` {#fixture_function}\n\n`def fixture_function(arg_1: str, arg_2) -> Tester`\n"

print(actual)
assert actual == expected

### `fixture_function` {#fixture_function}

`def fixture_function(arg_1: str, arg_2) -> Tester`



In [ ]:
def __fixture_function__(arg_1: str, arg_2) -> "Tester":
    pass


actual = _get_symbol_definition(__fixture_function__)
expected = "### `__fixture_function__` {#fixture_function}\n\n`def __fixture_function__(arg_1: str, arg_2) -> Tester`\n"

print(actual)
assert actual == expected

### `__fixture_function__` {#fixture_function}

`def __fixture_function__(arg_1: str, arg_2) -> Tester`



In [ ]:
def fixture_function(
    arg_1: int,
    arg_2: str = "default_string",
    arg_3: Dict[str, int] = {},
    arg_4: Optional[float] = None,
    arg_5: Tuple[int, str, float] = (1, "string", 2.0),
    arg_6: List[Union[int, str]] = [1, "string"],
    arg_7: Set[int] = {1, 2, 3},
    arg_8: Union[int, str] = "string",
) -> None:
    pass


actual = _get_symbol_definition(fixture_function)
expected = "### `fixture_function` {#fixture_function}\n\n`def fixture_function(arg_1: int, arg_2: str = 'default_string', arg_3: Dict[str, int] = {}, arg_4: Optional[float] = None, arg_5: Tuple[int, str, float] = (1, 'string', 2.0), arg_6: List[Union[int, str]] = [1, 'string'], arg_7: Set[int] = {1, 2, 3}, arg_8: Union[int, str] = 'string') -> None`\n"

print(actual)
assert actual == expected

### `fixture_function` {#fixture_function}

`def fixture_function(arg_1: int, arg_2: str = 'default_string', arg_3: Dict[str, int] = {}, arg_4: Optional[float] = None, arg_5: Tuple[int, str, float] = (1, 'string', 2.0), arg_6: List[Union[int, str]] = [1, 'string'], arg_7: Set[int] = {1, 2, 3}, arg_8: Union[int, str] = 'string') -> None`



In [ ]:
# | export


def _get_formatted_docstring_for_symbol(
    symbol: Union[types.FunctionType, Type[Any]]
) -> str:
    """Recursively parses and get formatted docstring of a symbol.

    Args:
        symbol: A Python class or function object to parse the docstring for.

    Returns:
        A formatted docstring of the symbol and its members.

    """

    def traverse(symbol: Union[types.FunctionType, Type[Any]], contents: str) -> str:
        """Recursively traverse the members of a symbol and append their docstrings to the provided contents string.

        Args:
            symbol: A Python class or function object to parse the docstring for.
            contents: The current formatted docstrings.

        Returns:
            The updated formatted docstrings.

        """
        for x, y in getmembers(symbol):
            if not x.startswith("_") or x.endswith("__"):
                if isfunction(y) and y.__doc__ is not None:
                    contents += f"{_get_symbol_definition(y)}\n{_docstring_to_markdown(y.__doc__)}"
                elif isclass(y) and not x.startswith("__") and y.__doc__ is not None:
                    contents += f"{_get_symbol_definition(y)}\n{_docstring_to_markdown(y.__doc__)}"
                    contents = traverse(y, contents)
        return contents

    contents = (
        f"{_get_symbol_definition(symbol)}\n{_docstring_to_markdown(symbol.__doc__)}"
        if symbol.__doc__ is not None
        else ""
    )
    if isclass(symbol):
        contents = traverse(symbol, contents)
    return contents

In [ ]:
def fixture_function(
    arg_1: str,
    arg_2: Union[List[str], str],
    arg_3: Optional[int],
    arg_4: Optional[str] = None,
) -> str:
    """This is a one line description for the function

    Args:
        arg_1: Argument 1
        arg_2: Argument 2
        arg_3: Argument 3
        arg_4: Argument 4

    Returns:
        The concatinated string
    """
    pass


expected = """### `fixture_function` {#fixture_function}\n\n`def fixture_function(arg_1: str, arg_2: Union[List[str], str], arg_3: Optional[int], arg_4: Optional[str] = None) -> str`

This is a one line description for the function

**Parameters**:
- `arg_1`: Argument 1
- `arg_2`: Argument 2
- `arg_3`: Argument 3
- `arg_4`: Argument 4

**Returns**:
- The concatinated string

"""

actual = _get_formatted_docstring_for_symbol(fixture_function)
print(actual)

assert actual == expected

### `fixture_function` {#fixture_function}

`def fixture_function(arg_1: str, arg_2: Union[List[str], str], arg_3: Optional[int], arg_4: Optional[str] = None) -> str`

This is a one line description for the function

**Parameters**:
- `arg_1`: Argument 1
- `arg_2`: Argument 2
- `arg_3`: Argument 3
- `arg_4`: Argument 4

**Returns**:
- The concatinated string




In [ ]:
class Vehicle:
    """This is a docstring for the class"""

    def __init__(self, brand: str, model: str, type: str):
        """Constructor

        Args:
            brand: Name of the brand
            model: Name of the model
            type: Model type
        """
        self.brand = brand
        self.model = model
        self.type = type
        self.gas_tank_size = 14
        self.fuel_level = 0

    def fuel_up(self):
        """Fuel up"""
        self.fuel_level = self.gas_tank_size
        print("Gas tank is now full.")

    def drive(self):
        """Drive"""
        print(f"The {self.model} is now driving.")


expected = """
This is a docstring for the class

### `__init__` {#init}

`def __init__(self, brand: str, model: str, type: str) -> None`

Constructor

**Parameters**:
- `brand`: Name of the brand
- `model`: Name of the model
- `type`: Model type

### `drive` {#drive}

`def drive(self) -> None`

Drive

### `fuel_up` {#fuel_up}

`def fuel_up(self) -> None`

Fuel up

"""

actual = _get_formatted_docstring_for_symbol(Vehicle)
print(actual)

assert actual == expected


This is a docstring for the class

### `__init__` {#init}

`def __init__(self, brand: str, model: str, type: str) -> None`

Constructor

**Parameters**:
- `brand`: Name of the brand
- `model`: Name of the model
- `type`: Model type

### `drive` {#drive}

`def drive(self) -> None`

Drive

### `fuel_up` {#fuel_up}

`def fuel_up(self) -> None`

Fuel up




In [ ]:
class Outer:
    """Outer Class"""

    def __init__(self):
        """Outer class constructor"""
        ## instantiating the 'Inner' class
        self.inner = self.Inner()

    def reveal(self):
        """Reveal function"""
        ## calling the 'Inner' class function display
        self.inner.inner_display("Calling Inner class function from Outer class")

    class Inner:
        """Inner Class"""

        def inner_display(self, msg):
            """Inner display"""
            print(msg)

    class Inner2:
        """Inner2 Class"""

        def inner_display_2(self, msg):
            """Inner display_2"""
            print(msg)


expected = """
Outer Class


Inner Class

### `inner_display` {#inner_display}

`def inner_display(self, msg) -> None`

Inner display


Inner2 Class

### `inner_display_2` {#inner_display_2}

`def inner_display_2(self, msg) -> None`

Inner display_2

### `__init__` {#init}

`def __init__(self) -> None`

Outer class constructor

### `reveal` {#reveal}

`def reveal(self) -> None`

Reveal function

"""

actual = _get_formatted_docstring_for_symbol(Outer)
print(actual)

assert actual == expected


Outer Class


Inner Class

### `inner_display` {#inner_display}

`def inner_display(self, msg) -> None`

Inner display


Inner2 Class

### `inner_display_2` {#inner_display_2}

`def inner_display_2(self, msg) -> None`

Inner display_2

### `__init__` {#init}

`def __init__(self) -> None`

Outer class constructor

### `reveal` {#reveal}

`def reveal(self) -> None`

Reveal function




In [ ]:
# | export


def _convert_html_style_attribute_to_jsx(contents: str) -> str:
    """Converts the inline style attributes in an HTML string to JSX compatible format.

    Args:
        contents: A string containing an HTML document or fragment.

    Returns:
        A string with inline style attributes converted to JSX compatible format.
    """
    style_regex = re.compile(r'style="(.+?)"')
    style_matches = style_regex.findall(contents)

    for style_match in style_matches:
        style_dict = {}
        styles = style_match.split(";")
        for style in styles:
            key_value = style.split(":")
            if len(key_value) == 2:
                key = re.sub(
                    r"-(.)", lambda m: m.group(1).upper(), key_value[0].strip()
                )
                value = key_value[1].strip().replace("'", '"')
                style_dict[key] = value
        replacement = "style={{"
        for key, value in style_dict.items():
            replacement += f"{key}: '{value}', "
        replacement = replacement[:-2] + "}}"
        contents = contents.replace(f'style="{style_match}"', replacement)

    return contents

In [ ]:
fixtures = [
    {
        "input": """<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style="float:right; font-size:smaller">source</a> some text goes here <a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style="float:right; font-size:smaller">source</a>""",
        "expected": """<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a> some text goes here <a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a>""",
    },
    {
        "input": '<span style="color: red;">Test</span>',
        "expected": "<span style={{color: 'red'}}>Test</span>",
    },
    {
        "input": '<div style="background-color: blue; border: 1px solid black;">Test</div>',
        "expected": "<div style={{backgroundColor: 'blue', border: '1px solid black'}}>Test</div>",
    },
    {
        "input": '<span style="font-size: 1.2rem;">Test</span>',
        "expected": "<span style={{fontSize: '1.2rem'}}>Test</span>",
    },
    {
        "input": """<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace"></pre>""",
        "expected": """<pre style={{whiteSpace: 'pre', overflowX: 'auto', lineHeight: 'normal', fontFamily: 'Menlo,"DejaVu Sans Mono",consolas,"Courier New",monospace'}}></pre>""",
    },
]

for fixture in fixtures:
    actual = _convert_html_style_attribute_to_jsx(fixture["input"])
    print("*" * 120)
    print(actual)
    assert actual == fixture["expected"], fixture["expected"]

************************************************************************************************************************
<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a> some text goes here <a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a>
************************************************************************************************************************
<span style={{color: 'red'}}>Test</span>
************************************************************************************************************************
<div style={{backgroundColor: 'blue', border: '1px solid black'}}>Test</div>
************************************************************************************************************************
<span style={{fontSize: '1

In [ ]:
# | export


def _get_all_markdown_files_path(docs_path: Path) -> List[Path]:
    """Get all Markdown files in a directory and its subdirectories.

    Args:
        directory: The path to the directory to search in.

    Returns:
        A list of paths to all Markdown files found in the directory and its subdirectories.
    """
    markdown_files = [file_path for file_path in docs_path.glob("**/*.md")]
    return markdown_files

In [ ]:
with TemporaryDirectory() as d:
    module_name = "fastkafka"

    docs_path = Path(d) / "docusaurus" / "docs"
    docs_path.mkdir(parents=True)

    api_path = docs_path / "api"
    api_path.mkdir(parents=True)

    blog_path = docs_path / "blog"
    blog_path.mkdir(parents=True)

    nested_api_path = api_path / "fastKafka"
    nested_api_path.mkdir(parents=True)

    for p in [docs_path, api_path, blog_path, nested_api_path]:
        with open((p / "file.md"), "w") as f:
            f.write("sample text")

    actual = _get_all_markdown_files_path(docs_path)
    expected = [
        Path(docs_path) / "file.md",
        Path(api_path) / "file.md",
        Path(nested_api_path) / "file.md",
        Path(blog_path) / "file.md",
    ]

    print(actual)
    assert sorted(actual) == sorted(expected), expected

[Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp4mz5t6rc/docusaurus/docs/file.md'), Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp4mz5t6rc/docusaurus/docs/blog/file.md'), Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp4mz5t6rc/docusaurus/docs/api/file.md'), Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp4mz5t6rc/docusaurus/docs/api/fastKafka/file.md')]


In [ ]:
# | export


def _fix_special_symbols_in_html(contents: str) -> str:
    contents = contents.replace("”", '"')
    return contents

In [ ]:
fixture = """<a href="https://colab.research.google.com/github/airtai/fastkafka/blob/main/nbs/guides/Guide_00_FastKafka_Demo.ipynb" target=”_blank”>"""
expected = """<a href="https://colab.research.google.com/github/airtai/fastkafka/blob/main/nbs/guides/Guide_00_FastKafka_Demo.ipynb" target="_blank">"""

actual = _fix_special_symbols_in_html(fixture)
print(actual)
assert actual == expected

<a href="https://colab.research.google.com/github/airtai/fastkafka/blob/main/nbs/guides/Guide_00_FastKafka_Demo.ipynb" target="_blank">


In [ ]:
# | export

def _add_file_extension_to_link(url: str) -> str:
    """Add file extension to the last segment of a URL

    Args:
        url: A URL string.

    Returns:
        A string of the updated URL with a file extension added to the last segment of the URL.
    """
    segments = url.split("/#")[0].split("/")[-2:]
    return url.replace(f"/{segments[1]}", f"/{segments[1]}.md")

In [ ]:
fixture = "https://airtai.github.io/fastkafka/api/fastkafka/FastKafka/#fastkafka.FastKafka"
expected = "https://airtai.github.io/fastkafka/api/fastkafka/FastKafka.md/#fastkafka.FastKafka"

actual = _add_file_extension_to_link(fixture)
print(actual)
assert actual == expected

https://airtai.github.io/fastkafka/api/fastkafka/FastKafka.md/#fastkafka.FastKafka


In [ ]:
fixture = "https://airtai.github.io/fastkafka/api/fastkafka/testing/ApacheKafkaBroker/#fastkafka.testing.ApacheKafkaBroker"
expected = "https://airtai.github.io/fastkafka/api/fastkafka/testing/ApacheKafkaBroker.md/#fastkafka.testing.ApacheKafkaBroker"

actual = _add_file_extension_to_link(fixture)
print(actual)
assert actual == expected

https://airtai.github.io/fastkafka/api/fastkafka/testing/ApacheKafkaBroker.md/#fastkafka.testing.ApacheKafkaBroker


In [ ]:
fixture = "https://github.com/airtai/sample_fastkafka_with_redpanda"
expected = "https://github.com/airtai/sample_fastkafka_with_redpanda.md"

actual = _add_file_extension_to_link(fixture)
print(actual)
assert actual == expected

https://github.com/airtai/sample_fastkafka_with_redpanda.md


In [ ]:
# | export


def _fix_symbol_links(contents: str, dir_prefix: str, doc_host: str, doc_baseurl: str) -> str:
    """Fix symbol links in Markdown content.

    Args:
        contents: The Markdown content to search for symbol links.
        dir_prefix: Directory prefix to append in the relative URL.
        doc_host: The host URL for the documentation site.
        doc_baseurl: The base URL for the documentation site.

    Returns:
        str: The Markdown content with updated symbol links.
    """
    prefix = re.escape(urljoin(doc_host + "/", doc_baseurl))
    pattern = re.compile(rf"\[(.*?)\]\(({prefix}[^)]+)\)")
    matches = pattern.findall(contents)
    for match in matches:
        old_url = match[1]
        new_url = _add_file_extension_to_link(old_url).replace(
            "/api/", "/docs/api/"
        )
        dir_prefix = "./" if dir_prefix == "" else dir_prefix
        relative_url = dir_prefix + new_url.split("/docs/")[1]
        contents = contents.replace(old_url, relative_url)
    return contents

In [ ]:
fixture = """In the above example,
[`FastKafka`](https://airtai.github.io/fastkafka/api/fastkafka/FastKafka/#fastkafka.FastKafka)
[`FastKafka`](https://airtai.github.io/fastkafka/0.5.0/api/fastkafka/FastKafka/#fastkafka.FastKafka)
app is named as `kafka_app`
[`FastKafka`](https://airtai.github.io/fastkafka/0.5.0rc0/api/fastkafka/FastKafka/#fastkafka.FastKafka)"""

expected = """In the above example,
[`FastKafka`](../api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
[`FastKafka`](../api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
app is named as `kafka_app`
[`FastKafka`](../api/fastkafka/FastKafka.md/#fastkafka.FastKafka)"""

dir_prefix = "../"
doc_host="https://airtai.github.io"
doc_baseurl="/fastkafka"
actual = _fix_symbol_links(fixture, dir_prefix, doc_host, doc_baseurl)
print(actual)
assert actual == expected

In the above example,
[`FastKafka`](../api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
[`FastKafka`](../api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
app is named as `kafka_app`
[`FastKafka`](../api/fastkafka/FastKafka.md/#fastkafka.FastKafka)


In [ ]:
fixture = """In the above example,
[`FastKafka`](https://airtai.github.io/fastkafka/api/fastkafka/FastKafka/#fastkafka.FastKafka)
[`FastKafka`](https://airtai.github.io/fastkafka/0.5.0/api/fastkafka/FastKafka/#fastkafka.FastKafka)
app is named as `kafka_app`
[`FastKafka`](https://airtai.github.io/fastkafka/0.5.0rc0/api/fastkafka/FastKafka/#fastkafka.FastKafka)"""

expected = """In the above example,
[`FastKafka`](./api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
[`FastKafka`](./api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
app is named as `kafka_app`
[`FastKafka`](./api/fastkafka/FastKafka.md/#fastkafka.FastKafka)"""

dir_prefix = ""
doc_host="https://airtai.github.io"
doc_baseurl="/fastkafka"
actual = _fix_symbol_links(fixture, dir_prefix, doc_host, doc_baseurl)
print(actual)
assert actual == expected

In the above example,
[`FastKafka`](./api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
[`FastKafka`](./api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
app is named as `kafka_app`
[`FastKafka`](./api/fastkafka/FastKafka.md/#fastkafka.FastKafka)


In [ ]:
fixture = """The service can be tested using the
[`Tester`](https://airtai.github.io/fastkafka/api/fastkafka/testing/Tester/#fastkafka.testing.Tester)
[`Tester`](https://airtai.github.io/fastkafka/0.5.0/api/fastkafka/testing/Tester/#fastkafka.testing.Tester)
[`Tester`](https://airtai.github.io/fastkafka/dev/api/fastkafka/testing/Tester/#fastkafka.testing.Tester)
instance and we can start the Kafka
broker locally using the
[`ApacheKafkaBroker`](https://airtai.github.io/fastkafka/api/fastkafka/testing/ApacheKafkaBroker/#fastkafka.testing.ApacheKafkaBroker)."""

expected = """The service can be tested using the
[`Tester`](./api/fastkafka/testing/Tester.md/#fastkafka.testing.Tester)
[`Tester`](./api/fastkafka/testing/Tester.md/#fastkafka.testing.Tester)
[`Tester`](./api/fastkafka/testing/Tester.md/#fastkafka.testing.Tester)
instance and we can start the Kafka
broker locally using the
[`ApacheKafkaBroker`](./api/fastkafka/testing/ApacheKafkaBroker.md/#fastkafka.testing.ApacheKafkaBroker)."""

dir_prefix = ""
doc_host="https://airtai.github.io"
doc_baseurl="/fastkafka"
actual = _fix_symbol_links(fixture, dir_prefix, doc_host, doc_baseurl)
print(actual)
assert actual == expected

The service can be tested using the
[`Tester`](./api/fastkafka/testing/Tester.md/#fastkafka.testing.Tester)
[`Tester`](./api/fastkafka/testing/Tester.md/#fastkafka.testing.Tester)
[`Tester`](./api/fastkafka/testing/Tester.md/#fastkafka.testing.Tester)
instance and we can start the Kafka
broker locally using the
[`ApacheKafkaBroker`](./api/fastkafka/testing/ApacheKafkaBroker.md/#fastkafka.testing.ApacheKafkaBroker).


In [ ]:
fixture = """This is not a link to a symbol: https://www.google.com"""

expected = """This is not a link to a symbol: https://www.google.com"""

dir_prefix = ""
doc_host="https://airtai.github.io"
doc_baseurl="/fastkafka"
actual = _fix_symbol_links(fixture, dir_prefix, doc_host, doc_baseurl)
print(actual)
assert actual == expected

This is not a link to a symbol: https://www.google.com


In [ ]:
fixture = """A sample fastkafka-based library that uses Redpanda for testing, based
on this guide, can be found
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)"""

expected = """A sample fastkafka-based library that uses Redpanda for testing, based
on this guide, can be found
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)"""

dir_prefix = ""
doc_host="https://airtai.github.io"
doc_baseurl="/fastkafka"
actual = _fix_symbol_links(fixture, dir_prefix, doc_host, doc_baseurl)
print(actual)
assert actual == expected

A sample fastkafka-based library that uses Redpanda for testing, based
on this guide, can be found
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)


In [ ]:
fixture = """To learn more about Redpanda, please visit their
[website](https://redpanda.com/) or checkout this [blog
post](https://redpanda.com/blog/redpanda-vs-kafka-performance-benchmark)
comparing Redpanda and Kafka’s performance benchmarks."""

expected = """To learn more about Redpanda, please visit their
[website](https://redpanda.com/) or checkout this [blog
post](https://redpanda.com/blog/redpanda-vs-kafka-performance-benchmark)
comparing Redpanda and Kafka’s performance benchmarks."""

dir_prefix = ""
doc_host="https://airtai.github.io"
doc_baseurl="/fastkafka"
actual = _fix_symbol_links(fixture, dir_prefix, doc_host, doc_baseurl)
print(actual)
assert actual == expected

To learn more about Redpanda, please visit their
[website](https://redpanda.com/) or checkout this [blog
post](https://redpanda.com/blog/redpanda-vs-kafka-performance-benchmark)
comparing Redpanda and Kafka’s performance benchmarks.


In [ ]:
# | export

def _get_relative_url_prefix(docs_path: Path, sub_path: Path) -> str:
    """Returns a relative url prefix from a sub path to a docs path.

    Args:
        docs_path (Path): The docs directory path.
        sub_path (Path): The sub directory path.

    Returns:
        str: A string representing the relative path from the sub path to the docs path.

    Raises:
        ValueError: If the sub path is not a descendant of the docs path.
    """
    try:
        relative_path = sub_path.relative_to(docs_path)
    except ValueError:
        raise ValueError(f"{sub_path} is not a descendant of {docs_path}")
    
    return "../" * (len(relative_path.parts) - 1) if len(relative_path.parts) > 1 else ""

In [ ]:
docs_path = Path('docusaurus/docs')

sub_path = Path('docusaurus/docs/index.md')
actual = _get_relative_url_prefix(docs_path, sub_path) 
print(actual)
assert actual == ""

sub_path = Path('docusaurus/docs/guides/Guide_31_Using_redpanda_to_test_fastkafka.md')
actual = _get_relative_url_prefix(docs_path, sub_path)
print(actual)
assert actual == "../"

sub_path = Path('docusaurus/docs/guides/tutorial/fastkafka.md')
actual = _get_relative_url_prefix(docs_path, sub_path)
print(actual)
assert actual == "../../"

with pytest.raises(ValueError) as e:
    sub_path = Path('mkdocs/docs/guides/tutorial/fastkafka.md')
    _get_relative_url_prefix(docs_path, sub_path)


../
../../


In [ ]:
# | export


def fix_invalid_syntax_in_markdown(docs_path: str) -> None:
    """Fix invalid HTML syntax in markdown files and converts inline style attributes to JSX-compatible format.

    Args:
        docs_path: The path to the root directory to search for markdown files.
    """
    cfg = get_config()
    doc_host = cfg["doc_host"]
    doc_baseurl = cfg["doc_baseurl"]
    
    markdown_files = _get_all_markdown_files_path(Path(docs_path))
    for file in markdown_files:
        relative_url_prefix = _get_relative_url_prefix(Path(docs_path), file)
        contents = Path(file).read_text()

        contents = _convert_html_style_attribute_to_jsx(contents)
        contents = _fix_special_symbols_in_html(contents)
        contents = _fix_symbol_links(contents, relative_url_prefix, doc_host, doc_baseurl)

        file.write_text(contents)

In [ ]:
with TemporaryDirectory() as d:
    module_name = "fastkafka"

    docs_path = Path(d) / "docusaurus" / "docs"
    docs_path.mkdir(parents=True)

    api_path = docs_path / "api"
    api_path.mkdir(parents=True)

    blog_path = docs_path / "blog"
    blog_path.mkdir(parents=True)

    nested_api_path = api_path / "fastKafka"
    nested_api_path.mkdir(parents=True)

    for p in [docs_path, api_path, blog_path, nested_api_path]:
        with open((p / "file.md"), "w") as f:
            f.write(
                """source some text goes here Test and one more tag Test
[`FastKafka`](https://airtai.github.io/fastkafka/api/fastkafka/FastKafka/#fastkafka.FastKafka)
[`Tester`](https://airtai.github.io/fastkafka/api/fastkafka/testing/Tester/#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)
"""
            )

    fix_invalid_syntax_in_markdown(str(docs_path))
    expected = [
        """source some text goes here Test and one more tag Test
[`FastKafka`](./api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
[`Tester`](./api/fastkafka/testing/Tester.md/#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)
""",
        """source some text goes here Test and one more tag Test
[`FastKafka`](../api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
[`Tester`](../api/fastkafka/testing/Tester.md/#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)
""",
        """source some text goes here Test and one more tag Test
[`FastKafka`](../api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
[`Tester`](../api/fastkafka/testing/Tester.md/#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)
""",
        """source some text goes here Test and one more tag Test
[`FastKafka`](../../api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
[`Tester`](../../api/fastkafka/testing/Tester.md/#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)
""",
    ]

    for i, p in enumerate([docs_path, api_path, blog_path, nested_api_path]):
        with open((p / "file.md"), "r") as f:
            actual = f.read()
            print("*" * 120)
            print(actual)
            assert actual == expected[i]

************************************************************************************************************************
source some text goes here Test and one more tag Test
[`FastKafka`](./api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
[`Tester`](./api/fastkafka/testing/Tester.md/#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)

************************************************************************************************************************
source some text goes here Test and one more tag Test
[`FastKafka`](../api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
[`Tester`](../api/fastkafka/testing/Tester.md/#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)

************************************************************************************************************************
source some text goes here Test and one more tag Test
[`FastKafka`](../api/fastkafka/FastKafka.md/#fastkafka.FastKafka)
[

In [ ]:
# | export


def generate_markdown_docs(module_name: str, docs_path: str) -> None:
    """Generates Markdown documentation files for the symbols in the given module and save them to the given directory.

    Args:
        module_name: The name of the module to generate documentation for.
        docs_path: The path to the directory where the documentation files will be saved.
    """
    members_with_submodules = _get_submodules(module_name)
    symbols = _load_submodules(module_name, members_with_submodules)

    for symbol in symbols:
        content = f"## `{symbol.__module__}.{symbol.__name__}` {{#{symbol.__module__}.{symbol.__name__}}}\n\n"
        content += _get_formatted_docstring_for_symbol(symbol)
        target_file_path = (
            "/".join(f"{symbol.__module__}.{symbol.__name__}".split(".")) + ".md"
        )

        with open((Path(docs_path) / "api" / target_file_path), "w") as f:
            f.write(content)

In [ ]:
with TemporaryDirectory() as d:
    module_name = "fastkafka"

    docs_path = Path(d) / "docusaurus" / "docs"
    docs_path.mkdir(parents=True)

    api_path = docs_path / "api"
    api_path.mkdir(parents=True)

    members_with_submodules = _get_submodules(module_name)
    symbols = _load_submodules(module_name, members_with_submodules)
    for symbol in symbols:
        target_file_path = (
            "/".join(f"{symbol.__module__}.{symbol.__name__}".split(".")) + ".md"
        )
        (api_path / "/".join(f"{symbol.__module__}".split("."))).mkdir(
            parents=True, exist_ok=True
        )

        with open((api_path / target_file_path), "w") as f:
            f.write(f"Initial content in '{target_file_path}'")

        with open((api_path / target_file_path), "r") as f:
            contents = f.read()
            print(contents)
            assert f"Initial content in '{target_file_path}'" == contents, contents

    generate_markdown_docs(module_name, str(docs_path))

    print("*" * 100)
    for symbol in symbols:
        target_file_path = (
            "/".join(f"{symbol.__module__}.{symbol.__name__}".split(".")) + ".md"
        )
        (api_path / "/".join(f"{symbol.__module__}".split("."))).mkdir(
            parents=True, exist_ok=True
        )

        with open((api_path / target_file_path), "r") as f:
            contents = f.read()
            print(contents)
            assert f"Initial content in '{target_file_path}'" != contents, contents

Initial content in 'fastkafka/EventMetadata.md'
Initial content in 'fastkafka/FastKafka.md'
Initial content in 'fastkafka/KafkaEvent.md'
Initial content in 'fastkafka/testing/ApacheKafkaBroker.md'
Initial content in 'fastkafka/testing/LocalRedpandaBroker.md'
Initial content in 'fastkafka/testing/Tester.md'
****************************************************************************************************
## `fastkafka.EventMetadata` {#fastkafka.EventMetadata}


A class for encapsulating Kafka record metadata.

**Parameters**:
- `topic`: The topic this record is received from
- `partition`: The partition from which this record is received
- `offset`: The position of this record in the corresponding Kafka partition
- `timestamp`: The timestamp of this record
- `timestamp_type`: The timestamp type of this record
- `key`: The key (or `None` if no key is specified)
- `value`: The value
- `serialized_key_size`: The size of the serialized, uncompressed key in bytes
- `serialized_value_size`:

In [ ]:
# | export

def _parse_lines(lines: List[str]) -> Tuple[List[str], int]:
    """Parse a list of lines and return a tuple containing a list of filenames and an index indicating how many lines to skip.

    Args:
        lines: A list of strings representing lines of input text.

    Returns:
        A tuple containing a list of strings representing the filenames extracted
        from links in the lines and an integer representing the number of lines to skip.
    """
    index = next(
        (i for i, line in enumerate(lines) if not line.strip().startswith("- [")),
        len(lines),
    )
    return [line.split("(")[1][:-4] for line in lines[:index]], index

In [ ]:
fixture = ['            - [json_encoder](api/fastkafka/encoder/json_encoder.md)', '        - testing', '            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)', '            - [LocalRedpandaBroker](api/fastkafka/testing/LocalRedpandaBroker.md)', '            - [Tester](api/fastkafka/testing/Tester.md)']
expected = (['api/fastkafka/encoder/json_encoder'], 1)

actual = _parse_lines(fixture)
print(actual)

assert actual == expected, actual

(['api/fastkafka/encoder/json_encoder'], 1)


In [ ]:

fixture = ['            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)', '            - [LocalRedpandaBroker](api/fastkafka/testing/LocalRedpandaBroker.md)', '            - [Tester](api/fastkafka/testing/Tester.md)']
expected = (['api/fastkafka/testing/ApacheKafkaBroker', 'api/fastkafka/testing/LocalRedpandaBroker', 'api/fastkafka/testing/Tester'], 3)

actual = _parse_lines(fixture)
print(actual)

assert actual == expected, actual

(['api/fastkafka/testing/ApacheKafkaBroker', 'api/fastkafka/testing/LocalRedpandaBroker', 'api/fastkafka/testing/Tester'], 3)


In [ ]:
# | export

def _parse_section(text: str, ignore_first_line: bool = False) -> List[Any]:
    """Parse the given section contents and return a list of file names in the expected format.

    Args:
        text: A string representing the contents of a file.
        ignore_first_line: Flag indicating whether to ignore the first line extracting the section contents.

    Returns:
        A list of filenames in the expected format
    """
    pattern = r"\[.*?\]\((.*?)\)|\[(.*?)\]\[(.*?)\]"
    lines = text.split("\n")[1:] if ignore_first_line else text.split("\n")
    ret_val = []
    index = 0
    while index < len(lines):
        line = lines[index]
        match = re.search(pattern, line.strip())
        if match is not None:
            ret_val.append(match.group(1).split(".md")[0])
            index += 1
        elif line.strip() !="":
            value, skip_lines = _parse_lines(lines[index + 1:])
            ret_val.append({line.replace('-', '').strip(): value})
            index += skip_lines + 1
        else:
            index += 1
    return ret_val

In [ ]:
fixture = """    - fastkafka
        - [FastKafka](api/fastkafka/FastKafka.md)
        - [KafkaEvent](api/fastkafka/KafkaEvent.md)
        - encoder
            - [json_encoder](api/fastkafka/encoder/json_encoder.md)
        - testing
            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)
            - [LocalRedpandaBroker](api/fastkafka/testing/LocalRedpandaBroker.md)
            - [Tester](api/fastkafka/testing/Tester.md)
"""

expected = [
    "api/fastkafka/FastKafka",
    "api/fastkafka/KafkaEvent",
    {"encoder": ["api/fastkafka/encoder/json_encoder"]},
    {
        "testing": [
            "api/fastkafka/testing/ApacheKafkaBroker",
            "api/fastkafka/testing/LocalRedpandaBroker",
            "api/fastkafka/testing/Tester",
        ]
    }
]

ignore_first_line = True
actual = _parse_section(fixture, ignore_first_line)
display(actual)
assert actual == expected

['api/fastkafka/FastKafka',
 'api/fastkafka/KafkaEvent',
 {'encoder': ['api/fastkafka/encoder/json_encoder']},
 {'testing': ['api/fastkafka/testing/ApacheKafkaBroker',
   'api/fastkafka/testing/LocalRedpandaBroker',
   'api/fastkafka/testing/Tester']}]

In [ ]:
fixture = """    - Writing services
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
        - [@consumes basics](guides/Guide_21_Produces_Basics.md)
    - Testing
        - [Using Redpanda to test FastKafka](guides/Guide_31_Using_redpanda_to_test_fastkafka.md)
"""

expected = [
    {
        "Writing services": [
            "guides/Guide_11_Consumes_Basics",
            "guides/Guide_21_Produces_Basics",
        ],
    },
    {
        "Testing": ["guides/Guide_31_Using_redpanda_to_test_fastkafka"],
    },
]

actual = _parse_section(fixture)
display(actual)
assert actual == expected

[{'Writing services': ['guides/Guide_11_Consumes_Basics',
   'guides/Guide_21_Produces_Basics']},
 {'Testing': ['guides/Guide_31_Using_redpanda_to_test_fastkafka']}]

In [ ]:
# | export

def _get_section_from_markdown(markdown_text: str, section_header: str) -> Optional[str]:
    """Get the contents of the section header from the given markdown text

    Args:
        markdown_text: A string containing the markdown text to extract the section from.
        section_header: A string representing the header of the section to extract.

    Returns:
        A string representing the contents of the section header if the section header 
        is present in the markdown text, else None
    """
    pattern = re.compile(fr'^- {section_header}\n((?:\s+- .*\n)+)', re.M)
    match = pattern.search(markdown_text)
    return match.group(1) if match else None

In [ ]:
summary = """- [FastKafka](index.md)
- Guides
    - Writing services
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
    - Testing
        - [Using Redpanda to test FastKafka](guides/Guide_31_Using_redpanda_to_test_fastkafka.md)
- API
    - fastkafka
        - [FastKafka](api/fastkafka/FastKafka.md)
        - [KafkaEvent](api/fastkafka/KafkaEvent.md)
        - encoder
            - [json_encoder](api/fastkafka/encoder/json_encoder.md)
        - testing
            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)
            - [LocalRedpandaBroker](api/fastkafka/testing/LocalRedpandaBroker.md)
            - [Tester](api/fastkafka/testing/Tester.md)
- CLI
    - [fastkafka](cli/fastkafka.md)
    - [run_fastkafka_server_process](cli/run_fastkafka_server_process.md)
- [Releases](CHANGELOG.md)"""

section_header = "API"
expected = """    - fastkafka
        - [FastKafka](api/fastkafka/FastKafka.md)
        - [KafkaEvent](api/fastkafka/KafkaEvent.md)
        - encoder
            - [json_encoder](api/fastkafka/encoder/json_encoder.md)
        - testing
            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)
            - [LocalRedpandaBroker](api/fastkafka/testing/LocalRedpandaBroker.md)
            - [Tester](api/fastkafka/testing/Tester.md)
"""
actual = _get_section_from_markdown(summary, section_header)
# print(actual)
assert actual == expected

In [ ]:
section_header = "CLI"
expected = """    - [fastkafka](cli/fastkafka.md)
    - [run_fastkafka_server_process](cli/run_fastkafka_server_process.md)
"""
actual = _get_section_from_markdown(summary, section_header)
print(actual)
assert actual == expected

    - [fastkafka](cli/fastkafka.md)
    - [run_fastkafka_server_process](cli/run_fastkafka_server_process.md)



In [ ]:
section_header = "Guides"
expected = """    - Writing services
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
    - Testing
        - [Using Redpanda to test FastKafka](guides/Guide_31_Using_redpanda_to_test_fastkafka.md)
"""
actual = _get_section_from_markdown(summary, section_header)
print(actual)
assert actual == expected

    - Writing services
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
    - Testing
        - [Using Redpanda to test FastKafka](guides/Guide_31_Using_redpanda_to_test_fastkafka.md)



In [ ]:
section_header = "Invalid Section"
expected = None
actual = _get_section_from_markdown(summary, section_header)
print(actual)
assert actual == expected

None


In [ ]:
# | export


def generate_sidebar(
    summary_file: str = "./docusaurus/docs/SUMMARY.md",
    summary: str = "",
    target: str = "./docusaurus/sidebars.js",
) -> None:
    with open(summary_file, "r") as stream, open(target, "w") as target_stream:
        summary_contents = stream.read()
        
        guides_summary = _get_section_from_markdown(summary_contents, "Guides")
        parsed_guides = _parse_section(guides_summary) # type: ignore
        
        api_summary = _get_section_from_markdown(summary_contents, "API")
        parsed_api = _parse_section(api_summary, True) # type: ignore
        
        cli_summary = _get_section_from_markdown(summary_contents, "CLI")
        parsed_cli = _parse_section(cli_summary) # type: ignore

        target_stream.write(
            """module.exports = {
tutorialSidebar: [
    'index', {'Guides': 
    """
            + str(parsed_guides)
            + "},"
            + "{'API': ["
            + str(parsed_api)[1:-1]
            + "]},"
            + "{'CLI': "
            + str(parsed_cli)
            + "},"
            + """
    "LICENSE",
    "CONTRIBUTING",
    "CHANGELOG",
],
};"""
        )

In [ ]:
summary = """- [FastKafka](index.md)
- Guides
    - Writing services
        - [Lifespan Events](guides/Guide_05_Lifespan_Handler.md)
        - [Encoding and Decoding Kafka Messages with FastKafka](guides/Guide_07_Encoding_and_Decoding_Messages_with_FastKafka.md)
    - Testing
        - [Using Redpanda to test FastKafka](guides/Guide_31_Using_redpanda_to_test_fastkafka.md)
    - Documentation generation
        - [Deploy FastKafka docs to GitHub Pages](guides/Guide_04_Github_Actions_Workflow.md)
- API
    - fastkafka
        - [FastKafka](api/fastkafka/FastKafka.md)
        - [KafkaEvent](api/fastkafka/KafkaEvent.md)
        - encoder
            - [AvroBase](api/fastkafka/encoder/AvroBase.md)
            - [json_decoder](api/fastkafka/encoder/json_decoder.md)
            - [json_encoder](api/fastkafka/encoder/json_encoder.md)
        - testing
            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)
- CLI
    - [fastkafka](cli/fastkafka.md)
    - [run_fastkafka_server_process](cli/run_fastkafka_server_process.md)
- [Releases](CHANGELOG.md)"""

with TemporaryDirectory() as directory:
    with open(directory + "/SUMMARY.md", "w") as stream:
        stream.write(summary)

    generate_sidebar(
        summary_file=directory + "/SUMMARY.md", target=directory + "/test.js"
    )

    with open(directory + "/test.js") as stream:
        stream = stream.read()

print(stream)
assert (
    stream
    == """module.exports = {
tutorialSidebar: [
    'index', {'Guides': 
    [{'Writing services': ['guides/Guide_05_Lifespan_Handler', 'guides/Guide_07_Encoding_and_Decoding_Messages_with_FastKafka']}, {'Testing': ['guides/Guide_31_Using_redpanda_to_test_fastkafka']}, {'Documentation generation': ['guides/Guide_04_Github_Actions_Workflow']}]},{'API': ['api/fastkafka/FastKafka', 'api/fastkafka/KafkaEvent', {'encoder': ['api/fastkafka/encoder/AvroBase', 'api/fastkafka/encoder/json_decoder', 'api/fastkafka/encoder/json_encoder']}, {'testing': ['api/fastkafka/testing/ApacheKafkaBroker']}]},{'CLI': ['cli/fastkafka', 'cli/run_fastkafka_server_process']},
    "LICENSE",
    "CONTRIBUTING",
    "CHANGELOG",
],
};"""
), stream

module.exports = {
tutorialSidebar: [
    'index', {'Guides': 
    [{'Writing services': ['guides/Guide_05_Lifespan_Handler', 'guides/Guide_07_Encoding_and_Decoding_Messages_with_FastKafka']}, {'Testing': ['guides/Guide_31_Using_redpanda_to_test_fastkafka']}, {'Documentation generation': ['guides/Guide_04_Github_Actions_Workflow']}]},{'API': ['api/fastkafka/FastKafka', 'api/fastkafka/KafkaEvent', {'encoder': ['api/fastkafka/encoder/AvroBase', 'api/fastkafka/encoder/json_decoder', 'api/fastkafka/encoder/json_encoder']}, {'testing': ['api/fastkafka/testing/ApacheKafkaBroker']}]},{'CLI': ['cli/fastkafka', 'cli/run_fastkafka_server_process']},
    "LICENSE",
    "CONTRIBUTING",
    "CHANGELOG",
],
};
